In [ ]:
# default_exp mltypes

# Mltypes

In [ ]:
# hide
%load_ext autoreload
%autoreload 2

In [ ]:
#exporti
import warnings
import random
import uuid
import attr
from typing import List

In [ ]:
# hide
import ipytest
import pytest
ipytest.autoconfig(raise_on_error=True)

## Types

In [ ]:
#export
@attr.define(slots=False)
class Coordinate:
    x: int
    y: int


@attr.define(slots=False)
class BboxCoordinate(Coordinate):
    width: int
    height: int


@attr.define(slots=False)
class BboxVideoCoordinate(BboxCoordinate):
    id: str

    def bbox_coord(self) -> BboxCoordinate:
        return BboxCoordinate(*list(attr.asdict(self).values())[:4])

In [ ]:
#hide
from attr import asdict

bbox_coord = BboxCoordinate(*[1, 2, 3, 4])
assert asdict(bbox_coord) == {'x': 1, 'y': 2, 'width': 3, 'height': 4}

video_coord = BboxVideoCoordinate(1, 1, 1, 1, '1')
assert asdict(video_coord) == {'x': 1, 'y': 1, 'width': 1, 'height': 1, 'id': '1'}

assert video_coord.bbox_coord() == BboxCoordinate(*[1, 1, 1, 1])

In [ ]:
#export

# todo: use pydantic
class Input():
    """
    Abstract class to represent input
    """

    def __repr__(self):
        return f"Annotator Input type: {self.__class__.__name__}"


class Output():
    """
    Abstract class to represent input
    """

    def __repr__(self):
        return f"Annotator Output type: {self.__class__.__name__}"

# Image classes

In [ ]:
#export
class InputImage(Input):
    """
    image_dir: string
        Directory of the image

    image_width: int
        Width of the image

    image_height: int
        Height of the image
    """

    def __init__(
        self,
        image_dir: str = 'pics',
        image_width: int = 100,
        image_height: int = 100,
        fit_canvas: bool = False
    ):
        self.width = image_width
        self.height = image_height
        self.dir = image_dir
        self.fit_canvas = fit_canvas

        if fit_canvas:
            warnings.warn("Image size will be ignored since fit_canvas is activated")

In [ ]:
%%ipytest
def test_it_warn_if_fit_canvas_is_activate_with_size():
    with warnings.catch_warnings(record=True) as w:
        inp_img = InputImage(image_width = 300, image_height = 300, fit_canvas=True)

    assert bool(w) is True

In [ ]:
%%ipytest
def test_it_doesnt_warn_if_fit_canvas_is_deactivate_with_size():
    with warnings.catch_warnings(record=True) as w:
        inp_img = InputImage(image_width = 300, image_height = 300, fit_canvas=False)

    assert bool(w) is False

In [ ]:
%%ipytest
def test_it_warn_if_fit_canvas_is_activate_with_size_none():
    with warnings.catch_warnings(record=True) as w:
        inp_img = InputImage(image_width=None, image_height=None, fit_canvas=True)
    assert bool(w) is True

In [ ]:
%%ipytest
def test_it_doesnt_warn_if_fit_canvas_is_deactivate_with_size_none():
    with warnings.catch_warnings(record=True) as w:
        inp_img = InputImage(image_width=None, image_height=None, fit_canvas=False)
    assert bool(w) is False

In [ ]:
# hide
imz = InputImage()
imz.dir

In [ ]:
#export
class OutputImageLabel(Output):
    """
     Will handle the image output.
     If no `label_dir` is specified, it should generate randomized one.

    """

    def __init__(self, label_dir=None, label_width=50, label_height=50):
        self.width = label_width
        self.height = label_height

        if label_dir is None:
            self.dir = 'class_autogenerated_' + ''.join(random.sample(str(uuid.uuid4()), 5))
        else:
            self.dir = label_dir

In [ ]:
#export
class OutputLabel(Output):
    def __init__(self, class_labels: List[str], label_width=50, label_height=50):
        self.width = label_width
        self.height = label_height
        self.class_labels = class_labels

In [ ]:
# hide

# label dir exists
lblz = OutputImageLabel(label_dir='class_images')
assert lblz.dir == 'class_images'

In [ ]:
# hide

# no label dir, should generate randomized name
lblz = OutputImageLabel()
assert 'class_autogenerated_' in lblz.dir

In [ ]:
#export
class OutputImageBbox(Output):
    def __init__(self, classes: List[str] = None):
        self.classes = classes or []
        self.drawing_enabled = True

In [ ]:
#export
class OutputVideoBbox(OutputImageBbox):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.drawing_enabled = True
        self.drawing_trajectory_enabled = True

In [ ]:
#export
class OutputGridBox(Output):
    pass

In [ ]:
#export
class NoOutput(Output):
    pass

In [ ]:
#hide
from nbdev.export import notebook2script
notebook2script()